# Data Preparation

## Packages

In [1]:
import os
import winsound
import numpy as np
import pandas as pd
import datetime as dt
from tqdm import tqdm
tqdm.pandas()
from multiprocessing import  Pool
import time

import math, collections
from scipy.stats import linregress

from matplotlib import pyplot as plt
from matplotlib.pyplot import figure

import warnings
warnings.filterwarnings('ignore')

In [2]:
def chunk_ticks(df, number_of_ticks):   
    global data
    
    df['tick'] = (df['Bid'] + df['Ask'])/2
    df['spread'] = df['Ask'] - df['Bid']
    df = df[['tick', 'spread']]
    
    temp_df = pd.DataFrame()
    spread_cls = []
    spread_avg = []
    
    tick_opn = []
    tick_hig = []
    tick_low = []
    tick_cls = []
    tick_avg = []
    tick_sd = []
    tick_ema_10 = []
    tick_ema_25 = []
    tick_ema_50 = []
    tick_ema_75 = []
    tick_ema = []
    
    for i in tqdm(range(0,len(df),number_of_ticks)):
        tick_list = list(df['tick'][i:i+number_of_ticks])
        spread_list = list(df['spread'][i:i+number_of_ticks])      
        
        spread_cls.append(spread_list[-1])
        spread_avg.append(np.mean(spread_list))

        tick_opn.append(tick_list[0])
        tick_hig.append(np.max(tick_list))
        tick_low.append(np.min(tick_list))
        tick_cls.append(tick_list[-1])
        tick_avg.append(np.mean(tick_list))  
        tick_sd.append(np.std(tick_list))
        
        l = tick_list[-(int(len(tick_list)*0.10)):]
        tick_ema_10.append(list(pd.DataFrame(l).ewm(span=len(l)).mean()[0])[len(l) - 1])        
        
        l = tick_list[-(int(len(tick_list)*0.25)):]
        tick_ema_25.append(list(pd.DataFrame(l).ewm(span=len(l)).mean()[0])[len(l) - 1])        
        
        l = tick_list[-(int(len(tick_list)*0.50)):]
        tick_ema_50.append(list(pd.DataFrame(l).ewm(span=len(l)).mean()[0])[len(l) - 1])        
        
        l = tick_list[-(int(len(tick_list)*0.75)):]
        tick_ema_75.append(list(pd.DataFrame(l).ewm(span=len(l)).mean()[0])[len(l) - 1])        
        
        tick_ema.append(list(pd.DataFrame(tick_list).ewm(span=len(tick_list)).mean()[0])[len(tick_list) - 1])        
        
    temp_df['spread_avg'] = spread_avg  
    temp_df['spread_cls'] = spread_cls  

    temp_df['tick_opn'] = tick_opn
    temp_df['tick_high'] = tick_hig
    temp_df['tick_low'] = tick_low
    temp_df['tick_cls'] = tick_cls
    temp_df['tick_avg'] = tick_avg  
    temp_df['tick_sd'] = tick_sd
    
    temp_df['tick_ema_10'] = tick_ema_10  
    temp_df['tick_ema_25'] = tick_ema_25  
    temp_df['tick_ema_50'] = tick_ema_50  
    temp_df['tick_ema_75'] = tick_ema_75  
    temp_df['tick_ema'] = tick_ema  

    return(temp_df)

In [3]:
def before_sma():
    global data    
    data['ssma_list'].append(val)    
    return()

def after_sma():
    global data
    
    data['ssma_list'].popleft()
    data['ssma_list'].append(val)
    data['sema'] = list(pd.DataFrame(list(data['ssma_list'])).ewm(span=data['sma_len']).mean()[0])[data['sma_len'] - 1]
    
    if len(data['sema_ready']) < 2:
        data['sema_ready'].append(data['sema'])
        data['sema_diff'] = np.nan

    elif len(data['sema_ready']) > 1:
        data['sema_ready'].popleft()
        data['sema_ready'].append(data['sema'])
        data['sema_diff'] = data['sema_ready'][-1] - data['sema_ready'][len(data['sema_ready'])-2]
    
    return()

def before_lma():
    global data    
    data['lsma_list'].append(val)    
    return()

def after_lma():
    global data
    
    data['lsma_list'].popleft()
    data['lsma_list'].append(val)
    data['lema'] = list(pd.DataFrame(list(data['lsma_list'])).ewm(span=data['lma_len']).mean()[0])[data['lma_len'] - 1]
    
    if len(data['lema_ready']) < 2:
        data['lema_ready'].append(data['lema'])
        data['lema_diff'] = np.nan

    elif len(data['lema_ready']) > 1:
        data['lema_ready'].popleft()
        data['lema_ready'].append(data['lema'])
        data['lema_diff'] = data['lema_ready'][-1] - data['lema_ready'][len(data['lema_ready'])-2]
    
    return()

In [4]:
def roll_sma(ssma_list):
    global data
    sema_val = list(pd.DataFrame(ssma_list).ewm(span=data['sma_len']).mean()[0])[data['sma_len'] - 1]    
    return(sema_val)

def roll_lma(lsma_list):
    global data
    lema_val = list(pd.DataFrame(ssma_list).ewm(span=data['lma_len']).mean()[0])[data['lma_len'] - 1]    
    return(lema_val)

In [5]:
def get_slope_s(y_axis):
    global data
    x_axis = []
    for i in range(data['sma_len']):
        x_axis.append(1 + ((i+1) * 0.0001 * 0.1))
    
    slope_tick, intercept, _, _, _ = linregress(x_axis, y_axis)
    slope_tick = math.degrees(math.atan(slope_tick))

    return(slope_tick)

def get_slope_l(y_axis):
    global data
    x_axis = []
    for i in range(data['lma_len']):
        x_axis.append(1 + ((i+1) * 0.0001 * 0.1))
    
    slope_tick, intercept, _, _, _ = linregress(x_axis, y_axis)
    slope_tick = math.degrees(math.atan(slope_tick))
    
    return(slope_tick)

## File paths

In [6]:
year = 2018
source_file_path = f'data/tick_{year}.csv'
path, file_name = os.path.split(source_file_path)

target_file_name = 'tab_'+file_name
target_file_path = os.path.join(path, target_file_name)

chunk_file_name = 'chunk_'+file_name
chunk_file_path = os.path.join(path, chunk_file_name)

print(f'source_file_path : {source_file_path}')
print(f'chunk_file_path : {chunk_file_path}')
print(f'target_file_path : {target_file_path}')

source_file_path : data/tick_2018.csv
chunk_file_path : data\chunk_tick_2018.csv
target_file_path : data\tab_tick_2018.csv


## Read data

In [7]:
data = {}
diff_col = 'sema'
data['pip_diff'] = 0.00002

data['number_of_ticks'] = 100

data['sma_len'] = 20
data['lma_len'] = 50

data['rsi_window'] = 14

In [8]:
%%time
df = pd.read_csv(source_file_path, nrows=100000)
#df = pd.read_csv(source_file_path)
df.head()

Wall time: 70.8 ms


,DateTime,Bid,Ask,Volume
0,20180101 22:00:08.661,1.20102,1.20143,2
1,20180101 22:00:08.895,1.20102,1.20148,2
2,20180101 22:00:10.634,1.20102,1.20147,2
3,20180101 22:00:11.223,1.20102,1.20148,2
4,20180101 22:00:29.530,1.20102,1.20145,2


In [9]:
%%time

df = chunk_ticks(df, data['number_of_ticks'])
df.to_csv(chunk_file_path, index = False)
print(f'Records : {len(df)}')

df = pd.read_csv(chunk_file_path)

100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:05<00:00, 194.73it/s]


Records : 1000
Wall time: 5.19 s


In [11]:
%%time

df['diff_next_high'] = df['tick_high'].shift(-1) - df['tick_cls']
df['diff_next_low'] = df['tick_low'].shift(-1) - df['tick_cls']

df['dir'] = 'same'
df.loc[abs(df['diff_next_high']) > abs(df['diff_next_low']), 'dir'] = 'high'
df.loc[abs(df['diff_next_low']) > abs(df['diff_next_high']), 'dir'] = 'low'

df['direction'] = 'same'
df.loc[(df['dir'] == 'high') & (df['diff_next_high'] >= 0.0002), 'direction'] = 'increase'
df.loc[(df['dir'] == 'high') & (df['diff_next_high'] <= -0.0002), 'direction'] = 'decrease'

df.loc[(df['dir'] == 'low') & (df['diff_next_low'] >= 0.0002), 'direction'] = 'increase'
df.loc[(df['dir'] == 'low') & (df['diff_next_low'] <= -0.0002), 'direction'] = 'decrease'

Wall time: 14 ms


In [12]:
%%time
col_list = ['tick_opn', 'tick_high', 'tick_low', 'tick_cls', 'tick_avg', 'tick_ema', 'spread_cls', 'tick_ema_10', 'tick_ema_25', 'tick_ema_50', 'tick_ema_75']
for i, val in enumerate(col_list):    
    df[val+'_diff'] =  df[val].diff()
    del df[val]    

Wall time: 11 ms


In [15]:
data['rs_max'] = 1e6
df['gain'] = 0
df['loss'] = 0
df['gain'].loc[df['tick_avg_diff'] > 0] = abs(df['tick_avg_diff'])
df['loss'].loc[df['tick_avg_diff'] < 0] = abs(df['tick_avg_diff'])
df['avg_gain'] = df['gain'].rolling(window=data['rsi_window']).mean()
df['avg_loss'] = df['loss'].rolling(window=data['rsi_window']).mean()
df['rs'] = df['avg_gain']/df['avg_loss']
df['rs'] = df['rs'].where(df['rs'] <= data['rs_max'], data['rs_max']) 
df['rsi'] = 100 - (100 / (df['rs'] + 1))

In [16]:
df.head()

,spread_avg,tick_sd,diff_next_high,diff_next_low,dir,direction,tick_opn_diff,tick_high_diff,tick_low_diff,tick_cls_diff,...,tick_ema_10_diff,tick_ema_25_diff,tick_ema_50_diff,tick_ema_75_diff,gain,loss,avg_gain,avg_loss,rs,rsi
0,0.000491,0.000183,0.000515,0.000045,high,increase,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.000000,0.000000,NaN,NaN,1000000.0,99.9999
1,0.000242,0.000089,0.000030,-0.000380,low,decrease,-0.000525,-0.000085,0.000235,0.000465,...,0.000507,4.573955e-04,0.000402,0.000381,0.000289,0.000000,NaN,NaN,1000000.0,99.9999
2,0.000302,0.000145,0.000025,-0.000105,low,same,0.000420,-0.000020,0.000040,-0.000355,...,-0.000347,-3.155078e-04,-0.000287,-0.000278,0.000000,0.000172,NaN,NaN,1000000.0,99.9999
3,0.000256,0.000036,0.000025,0.000000,high,same,-0.000345,-0.000360,-0.000080,-0.000055,...,-0.000047,-4.611794e-05,-0.000047,-0.000054,0.000000,0.000129,NaN,NaN,1000000.0,99.9999
4,0.000250,0.000006,0.000070,-0.000030,high,same,-0.000055,-0.000055,0.000050,0.000010,...,0.000002,9.998953e-07,0.000004,0.000007,0.000000,0.000006,NaN,NaN,1000000.0,99.9999


In [17]:
df.head(50)

,spread_avg,tick_sd,diff_next_high,diff_next_low,dir,direction,tick_opn_diff,tick_high_diff,tick_low_diff,tick_cls_diff,...,tick_ema_10_diff,tick_ema_25_diff,tick_ema_50_diff,tick_ema_75_diff,gain,loss,avg_gain,avg_loss,rs,rsi
0,0.000491,0.000183,0.000515,0.000045,high,increase,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.000000,0.000000,NaN,NaN,1000000.000000,99.999900
1,0.000242,0.000089,0.000030,-0.000380,low,decrease,-0.000525,-0.000085,0.000235,0.000465,...,0.000507,4.573955e-04,0.000402,0.000381,0.000289,0.000000,NaN,NaN,1000000.000000,99.999900
2,0.000302,0.000145,0.000025,-0.000105,low,same,0.000420,-0.000020,0.000040,-0.000355,...,-0.000347,-3.155078e-04,-0.000287,-0.000278,0.000000,0.000172,NaN,NaN,1000000.000000,99.999900
3,0.000256,0.000036,0.000025,0.000000,high,same,-0.000345,-0.000360,-0.000080,-0.000055,...,-0.000047,-4.611794e-05,-0.000047,-0.000054,0.000000,0.000129,NaN,NaN,1000000.000000,99.999900
4,0.000250,0.000006,0.000070,-0.000030,high,same,-0.000055,-0.000055,0.000050,0.000010,...,0.000002,9.998953e-07,0.000004,0.000007,0.000000,0.000006,NaN,NaN,1000000.000000,99.999900
5,0.000240,0.000019,0.000670,-0.000005,high,increase,-0.000005,0.000055,-0.000020,0.000015,...,0.000015,1.576827e-05,0.000009,0.000002,0.000000,0.000008,NaN,NaN,1000000.000000,99.999900
6,0.000216,0.000247,0.000375,-0.000045,high,increase,0.000010,0.000615,0.000040,0.000590,...,0.000600,6.015641e-04,0.000610,0.000577,0.000416,0.000000,NaN,NaN,1000000.000000,99.999900
7,0.000221,0.000119,0.000100,-0.000020,high,same,0.000595,0.000295,0.000550,0.000150,...,0.000142,1.814487e-04,0.000175,0.000180,0.000280,0.000000,NaN,NaN,1000000.000000,99.999900
8,0.000223,0.000025,0.000005,-0.000165,low,same,0.000140,-0.000125,0.000175,0.000085,...,0.000081,2.436458e-05,0.000016,0.000045,0.000105,0.000000,NaN,NaN,1000000.000000,99.999900
9,0.000112,0.000037,0.000045,-0.000185,low,same,0.000095,-0.000010,-0.000060,-0.000075,...,-0.000073,-6.071987e-05,-0.000061,-0.000067,0.000000,0.000073,NaN,NaN,1000000.000000,99.999900


In [ ]:
%%time
del_list = ['diff_next_high', 'diff_next_low', 'dir']
for i, val in enumerate(del_list):    
    del df[val]  

In [ ]:
col_order = [
 'spread_avg',
 'spread_cls_diff', 
 'tick_opn_diff',
 'tick_high_diff',
 'tick_low_diff',
 'tick_cls_diff',
 'tick_avg_diff',
 'tick_sd',
 'tick_ema_10_diff',
 'tick_ema_25_diff',
 'tick_ema_50_diff',
 'tick_ema_75_diff',
 'tick_ema_diff',
 'direction'
 ]

df = df[col_order]
df = df.dropna()

In [ ]:
df.head()

In [ ]:
df['direction'].value_counts(normalize = True)

In [ ]:
df['avg_diff'] = abs(np.round(df['tick_avg'].diff(),4))
df['act_diff'] = abs(np.round(df['tick_act'].diff(),4))
df.head()

df['avg_diff'].value_counts(normalize = True)

df['act_diff'].value_counts(normalize = True)

In [ ]:
%%time

data['rs_max'] = 1e6

data['ssma_list'] = collections.deque([])
data['lsma_list'] = collections.deque([])
data['sema_ready'] = collections.deque([])
data['lema_ready'] = collections.deque([])
df['sema'] = ''
df['lema'] = ''
df['sema_diff'] = ''
df['lema_diff'] = ''


# RSI -----------------------------
df['diff'] = df['tick_avg'].diff()
df['gain'] = 0
df['loss'] = 0
df['gain'].loc[df['diff'] > 0] = abs(df['diff'])
df['loss'].loc[df['diff'] < 0] = abs(df['diff'])
df['avg_gain'] = df['gain'].rolling(window=data['rsi_window']).mean()
df['avg_loss'] = df['loss'].rolling(window=data['rsi_window']).mean()
df['rs'] = df['avg_gain']/df['avg_loss']
df['rs'] = df['rs'].where(df['rs'] <= data['rs_max'], data['rs_max']) 
df['rsi'] = 100 - (100 / (df['rs'] + 1))

# Simple Moving Averages ------------------
df['ssma'] = df['tick_avg'].rolling(window=data['sma_len']).mean()
df['ssma_diff'] = df['ssma'].diff()
df['lsma'] = df['tick_avg'].rolling(window=data['lma_len']).mean()
df['lsma_diff'] = df['lsma'].diff()
df['sma_diff'] = df['ssma'] - df['lsma']

df['max_tick'] = df['tick_avg'].rolling(window=data['sma_len']).max()
df['min_tick'] = df['tick_avg'].rolling(window=data['sma_len']).min()

df['max_gap'] = df['max_tick'] -  df['tick_avg']
df['min_gap'] = df['min_tick'] - df['tick_avg']

df['tick_high_diff'] = df['tick_high'] - df['tick_avg']
df['tick_low_diff'] = df['tick_avg'] - df['tick_low']

In [ ]:
%%time
# Emas ----------------
df['sema'] = df['tick_avg'].rolling(window=data['sma_len']).progress_apply(roll_sma)
df['lema'] = df['tick_avg'].rolling(window=data['lma_len']).progress_apply(roll_sma)

df['sema_diff'] = df['sema'].diff()
df['lema_diff'] = df['lema'].diff()

%%time
# Slopes -----------------------------
df['small_sema_slope'] = df['sema'].rolling(window=data['sma_len']).progress_apply(get_slope_s)
df['long_sema_slope'] = df['sema'].rolling(window=data['lma_len']).progress_apply(get_slope_l)

In [ ]:
%%time

df['ema_diff'] = df['sema'] - df['lema']

# Direction -------------------------
df['direction'] = 'same'
df['direction'].loc[df[diff_col].shift(-1) - df[diff_col] > data['pip_diff']] = 'increase'
df['direction'].loc[df[diff_col].shift(-1) - df[diff_col] < -data['pip_diff']] = 'decrease'

# Remove NaNs ------------------------
del df['gain']
del df['loss']
df = df.dropna()
df = df.reset_index(drop=True)
print(f'Total records : {len(df)}')

## Write data to csv

In [ ]:
%%time
df.to_csv(target_file_path, index = False)
winsound.PlaySound('C:\\Windows\\Media\\tada.wav', winsound.SND_ASYNC)

## Print Report

In [ ]:
g= df['direction']
print(pd.concat([g.value_counts(), g.value_counts(normalize=True).mul(100)],axis=1, keys=('counts','percentage')))
df.head(5)

In [ ]:
inc = df[df['direction'] == 'increase']['diff']
dec = df[df['direction'] == 'decrease']['diff']

In [ ]:
pd.DataFrame(np.round(df['tick_low_diff'],4))['tick_low_diff'].value_counts()

In [ ]:
pd.DataFrame(np.round(df['tick_high_diff'],4))['tick_high_diff'].value_counts()